In [17]:
from pe_features import *
import csv, os, pefile
import pandas as pd 

result_file = "./test_data2_pe_features.csv"
file_path ="./sample_data"
files = os.listdir(file_path)

with open(result_file, 'w') as fp: 
    csv_fp = csv.writer(fp) 
    csv_fp.writerow(COLS) ,

    for filename in files: 
        fullname = os.path.join(file_path, filename) 
        try: 
            pe = pefile.PE(fullname) 
            row = extract_pe_features(pe, filename) 
            print("[+] {}". format(filename)) 
        except:
            row = NULL_ROW 
            row[0] = filename 
            print("[-] This file is not pe format! {}".format(filename))

        csv_fp.writerow(row)


[+] 03fc26330972ff3d59d9f21281b14c9c.vir
[+] 107440b4d3c67f405721cf37fd723212.vir
[+] 3e2f89c6a7a29eab827c77396ffaf02c.vir
[+] dc36522c933b70c6245a2f82e5404c8f.vir
[+] 9dc2c96cc84d6f7c4e9ebbabcbf88687.vir
[+] 18512afd7117bf5199947b542404e3e8.vir
[+] 3c1341a965226e92e2a28288ebe7f9fe.vir
[+] 23160df27a0c341122e006796aeabe97.vir
[+] 9c6e9451cbceb20d12b8af6f8881a246.vir
[+] 48e7debe0f70695a5d6b3e4d07a3b49e.vir
[+] c3e2f29993392a1d8a4c90b90222364a.vir
[-] This file is not pe format! f25f96953199aaa49f8f5864be4cf786.vir
[+] dc07db16958e1b4648183d69f92679c2.vir
[+] ad57e63a7b09bf98f60d03b95bf90f02.vir
[+] 33b5fb836887814e400a873e0b66dd7d.vir
[+] ea81cf8a146a8d5eec260cc460b9fe26.vir
[+] 5c2680ea90417d62756ac699b0c54af0.vir
[+] 0a7d4c082deff16b2224d1298d590bb1.vir
[+] c8820f9fc0672c1780e52b4c481682bc.vir
[+] d71bac17766bac45a4bea24cada30926.vir
[+] da227009517eeae29b4c775dbb2b6add.vir
[+] 04b170d31349b10cb99dd86dd76af169.vir
[+] 74077c3e366f9fb15a268babc8128dd8.vir
[+] 83d0852f2a964750617a63728

In [20]:
pe_result = pd.read_csv(result_file)
target = pd.read_csv("./sample_data_label.csv")
dataset = pe_result.join(target.set_index('filename')['label'],on='filename')
label = dataset['label']

data = dataset.drop(['filename', 'label'], axis = 1)
data = data.dropna(axis=1)
data

,e_magic,e_lfanew,e_minalloc,e_ovno,Signature,Machine,NumberOfSections,TimeDateStamp,PointerToSymbolTable,NumberOfSymbols,...,SectionAlignment,FileAlignment,SizeOfImage,SizeOfHeaders,CheckSum,Subsystem,NumberOfRvaAndSizes,CompareNumberOfSections,TotalNumberOfFunctionInIAT,TotalNumberOfFunctionInEAT
0,23117,184,0,0,17744,452,2,3843626401,0,0,...,4096,4096,12288,4096,64405,3,16,1,0,0
1,23117,248,0,0,17744,332,5,1468633372,0,0,...,4096,512,98304,1024,92993,2,16,1,28,0
2,23117,256,15,26,17744,332,9,1339084793,0,0,...,4096,512,1097728,1024,1102371,2,16,1,600,0
3,23117,216,0,0,17744,332,5,1260053452,0,0,...,4096,512,233472,1024,0,2,16,1,155,0
4,23117,200,0,0,17744,332,3,1451299490,0,0,...,4096,4096,81920,4096,122832,2,16,1,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,23117,256,15,26,17744,332,9,1339084793,0,0,...,4096,512,729088,1024,0,2,16,1,600,0
96,23117,232,0,0,17744,332,4,1433144509,0,0,...,4096,512,6324224,1024,6355790,2,16,1,45,0
97,23117,128,0,0,17744,332,11,1071417432,0,0,...,4096,512,217088,1024,0,2,16,1,3,0
98,23117,216,0,0,17744,332,4,1273920704,0,0,...,4096,512,20480,1024,0,2,16,1,27,5


In [45]:
from sklearn.preprocessing import StandardScaler

std_cls=StandardScaler()
std_cls.fit(data)
std_data = std_cls.transform(data)
col_name = data.columns
std_df = pd.DataFrame(data=std_data, columns=col_name)

from sklearn.preprocessing import MinMaxScaler
min_cls = MinMaxScaler()
min_cls.fit(data)
min_data = min_cls.transform(data)
min_df = pd.DataFrame(data=min_data, columns=col_name)

from sklearn.model_selection import train_test_split

tr_X, te_X, tr_y, te_y = train_test_split(std_df, label, test_size=0.2, random_state=2024)
tr_X.shape, te_X.shape, tr_y.shape, te_y.shape

((80, 28), (20, 28), (80,), (20,))

In [46]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(tr_X, tr_y)
model.score(te_X, te_y)

0.55

In [31]:
tr_X, te_X, tr_y, te_y = train_test_split(min_df, label, test_size=0.2, random_state=2024)
tr_X.shape, te_X.shape, tr_y.shape, te_y.shape

model = KNeighborsClassifier(n_neighbors=3)
model.fit(tr_X, tr_y)
model.score(te_X, te_y)

0.55